# **Day22**: *Sand Slabs*

## Part 1

Enough sand has fallen; it can finally filter water for Snow Island.

Well, almost.

The sand has been falling as large compacted bricks of sand, piling up to form an impressive stack here near the edge of Island Island. In order to make use of the sand to filter water, some of the bricks will need to be broken apart - nay, disintegrated - back into freely flowing sand.

The stack is tall enough that you'll have to be careful about choosing which bricks to disintegrate; if you disintegrate the wrong brick, large portions of the stack could topple, which sounds pretty dangerous.

The Elves responsible for water filtering operations took a snapshot of the bricks while they were still falling (your puzzle input) which should let you work out which bricks are safe to disintegrate. For example:

```
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9
```

Each line of text in the snapshot represents the position of a single brick at the time the snapshot was taken. The position is given as two x,y,z coordinates - one for each end of the brick - separated by a tilde (~). Each brick is made up of a single straight line of cubes, and the Elves were even careful to choose a time for the snapshot that had all of the free-falling bricks at integer positions above the ground, so the whole snapshot is aligned to a three-dimensional cube grid.

A line like 2,2,2~2,2,2 means that both ends of the brick are at the same coordinate - in other words, that the brick is a single cube.

Lines like 0,0,10~1,0,10 or 0,0,10~0,1,10 both represent bricks that are two cubes in volume, both oriented horizontally. The first brick extends in the x direction, while the second brick extends in the y direction.

A line like 0,0,1~0,0,10 represents a ten-cube brick which is oriented vertically. One end of the brick is the cube located at 0,0,1, while the other end of the brick is located directly above it at 0,0,10.

The ground is at z=0 and is perfectly flat; the lowest z value a brick can have is therefore 1. So, 5,5,1~5,6,1 and 0,2,1~0,2,5 are both resting on the ground, but 3,3,2~3,3,3 was above the ground at the time of the snapshot.

Because the snapshot was taken while the bricks were still falling, some bricks will still be in the air; you'll need to start by figuring out where they will end up. Bricks are magically stabilized, so they never rotate, even in weird situations like where a long horizontal brick is only supported on one end. Two bricks cannot occupy the same position, so a falling brick will come to rest upon the first other brick it encounters.

Here is the same example again, this time with each brick given a letter so it can be marked in diagrams:

```
1,0,1~1,2,1   <- A
0,0,2~2,0,2   <- B
0,2,3~2,2,3   <- C
0,0,4~0,2,4   <- D
2,0,5~2,2,5   <- E
0,1,6~2,1,6   <- F
1,1,8~1,1,9   <- G
```

At the time of the snapshot, from the side so the x axis goes left to right, these bricks are arranged like this:

```
 x
012
.G. 9
.G. 8
... 7
FFF 6
..E 5 z
D.. 4
CCC 3
BBB 2
.A. 1
--- 0
```

Rotating the perspective 90 degrees so the y axis now goes left to right, the same bricks are arranged like this:

```
 y
012
.G. 9
.G. 8
... 7
.F. 6
EEE 5 z
DDD 4
..C 3
B.. 2
AAA 1
--- 0
```

Once all of the bricks fall downward as far as they can go, the stack looks like this, where ? means bricks are hidden behind other bricks at that location:

```
 x
012
.G. 6
.G. 5
FFF 4
D.E 3 z
??? 2
.A. 1
--- 0
```

Again from the side:

```
 y
012
.G. 6
.G. 5
.F. 4
??? 3 z
B.C 2
AAA 1
--- 0
```

Now that all of the bricks have settled, it becomes easier to tell which bricks are supporting which other bricks:

- Brick A is the only brick supporting bricks B and C.
- Brick B is one of two bricks supporting brick D and brick E.
- Brick C is the other brick supporting brick D and brick E.
- Brick D supports brick F.
- Brick E also supports brick F.
- Brick F supports brick G.
- Brick G isn't supporting any bricks.

Your first task is to figure out which bricks are safe to disintegrate. A brick can be safely disintegrated if, after removing it, no other bricks would fall further directly downward. Don't actually disintegrate any bricks - just determine what would happen if, for each brick, only that brick were disintegrated. Bricks can be disintegrated even if they're completely surrounded by other bricks; you can squeeze between bricks if you need to.

In this example, the bricks can be disintegrated as follows:

- Brick A cannot be disintegrated safely; if it were disintegrated, bricks B and C would both fall.
- Brick B can be disintegrated; the bricks above it (D and E) would still be supported by brick C.
- Brick C can be disintegrated; the bricks above it (D and E) would still be supported by brick B.
- Brick D can be disintegrated; the brick above it (F) would still be supported by brick E.
- Brick E can be disintegrated; the brick above it (F) would still be supported by brick D.
- Brick F cannot be disintegrated; the brick above it (G) would fall.
- Brick G can be disintegrated; it does not support any other bricks.

So, in this example, 5 bricks can be safely disintegrated.

Figure how the blocks will settle based on the snapshot. Once they've settled, consider disintegrating a single brick; how many bricks could be safely chosen as the one to get disintegrated?

### Solution

In [41]:
from operator import attrgetter, itemgetter

def my_func(input_string):
    bricks = []
    z_max = 0
    for i, line in enumerate(input_string.splitlines()):
        start, end = line.split('~')
        start_position = np.array(tuple(map(int, start.split(','))))
        end_position = np.array(tuple(map(int, end.split(','))))
        z_max = max(z_max, start_position[2], end_position[2])
        bricks.append(Brick(i+1, start_position, end_position))   
    
    bricks = sorted(bricks, key=attrgetter('zmin'))
    elevations = {z:[] for z in range(1, z_max+1)}
    
    for brick in bricks:
        elevations[brick.zmin].append(brick)
        
    for z, bricks_list in elevations.items():
        # skip z = 1
        if z<2: continue
            
        # loop over bricks at current elevation
        for brick in bricks_list:
            destination_level = z
            move = True
            
            while move:
                lower_level = destination_level -1 
                
                # check if lower level is obstructed by ground
                if lower_level < 1:
                    move = False
                    
                # check if lower level is obstructed by bricks
                for other_brick in elevations[lower_level]:
                    if brick.intersect(other_brick): 
                        move = False
                        break
                
                if move:
                    destination_level -= 1
                
            
            # check if brick destination level is different than current level
            if destination_level < z:
                # move brick
                brick.move(dz=z - destination_level)
                elevations[z].remove(brick)
                elevations[destination_level].append(brick)
                
    print()     
        


In [21]:
import numpy as np


class Ground:
    positions = [np.array((i, j, 0)) for i in range(10) for j in range(10)]
    zmin = 0
    
    
class Brick:
    
    def __init__(self, brick_id, start_position, end_position) -> None:
        self.id = brick_id
        
        steps = max(abs(end_position - start_position))+1
        self.positions = np.linspace(start_position, end_position, steps)
        self.zmin = min(start_position[2], end_position[2])
        
    def __repr__(self) -> str:
        return f"Brick{self.id}"
        
    def intersect(self, positions):
        """Check if a list of positions intersect this brick's positions"""
        return any(np.array_equal(p1, p2) for p1 in positions for p2 in self.positions)
    
    def move(self, dz):
        """Update brick's positions"""
        for p in self.positions:
            p[2] += dz
            
        
        
        
    


In [22]:
def settle(bricks):
    change = False
    for current_brick in bricks:
        if type(current_brick) == Ground: continue 
        current_positions = current_brick.positions.copy()
        valid_position = True
        
        while valid_position:
            new_positions = current_positions + np.array((0, 0, -1))
            
            # check if valid position
            for other_brick in bricks:
                if type(other_brick) == Ground or other_brick is current_brick: 
                    continue 
                
                if other_brick.intersect(new_positions):
                    valid_position = False
                    break
                
                if not all(p[2]>0 for p in new_positions):
                    valid_position = False
                    break
                    
            if valid_position:
                current_positions = new_positions
                change = True
        
        current_brick.positions = current_positions
    
    return change

In [43]:
from operator import attrgetter, itemgetter


def my_func(input_string):
    ground = Ground()
    bricks = [ground]
    z_max = 0
    for i, line in enumerate(input_string.splitlines()):
        start, end = line.split('~')
        start_position = np.array(tuple(map(int, start.split(','))))
        end_position = np.array(tuple(map(int, end.split(','))))
        z_max = max(z_max, start_position[2], end_position[2])
        bricks.append(Brick(i+1, start_position, end_position))   
    
    bricks = sorted(bricks, key=attrgetter('zmin'))
    settle(bricks)
    print('settled')
    
    counter = 0
    for i, removed_brick in enumerate(bricks[1:]):
        print(f"{i}/{len(bricks)}")
        new_bricks = [brick for brick in bricks if brick is not removed_brick]
        changed = settle(new_bricks)
        if changed:
            counter += 1
    
    return len(bricks) - counter - 1
    
        
        
            
            

In [53]:
class Bricks:
    def __init__(self):
        self.index = {}          # key:value = position:index
        self.positions_dict = {}  # key:value = index:positions

    def order_positions_dict_by_lowest_z(self):
        self.positions_dict = {index: sorted(positions, key=lambda pos: pos[2]) 
                               for index, positions in self.positions_dict.items()}

    def __repr__(self):
        return f"Bricks"

# Example usage:
bricks = Bricks()

# Assuming self.positions_dict is populated
bricks.positions_dict = {
    1: [(1, 2, 3), (2, 3, 1), (3, 1, 2)],
    2: [(4, 5, 2), (5, 4, 3), (6, 6, 1)],
    # ... other index:positions entries
}

# Order self.positions_dict by the lowest z value
bricks.order_positions_dict_by_lowest_z()

# Print the ordered positions_dict
print(bricks.positions_dict)


{1: [(2, 3, 1), (3, 1, 2), (1, 2, 3)], 2: [(6, 6, 1), (4, 5, 2), (5, 4, 3)]}


In [59]:
# Your dictionary
your_dict = {
    'key1': [(1, 2, 3), (4, 5, 6), (7, 8, 9)],
    'key2': [(10, 11, 12), (13, 14, 15), (16, 17, 18)],
    'key3': [(19, 20, 1), (22, 23, 24), (25, 26, 27)]
}

# Function to get the minimum z value from a list of tuples
def get_min_z(tuple_list):
    return min(tuple_list, key=lambda x: x[2])[2]

# Sorting keys based on the minimum z value of their corresponding lists
sorted_keys = sorted(your_dict.keys(), key=lambda k: get_min_z(your_dict[k]))

print("Sorted keys based on the minimum z value:")
print(sorted_keys)


Sorted keys based on the minimum z value:
['key3', 'key1', 'key2']


In [45]:
import matplotlib.pyplot as pyplot


class Bricks:
    
    def __init__(self):
        self.index = {}      # key:value = position:index
        self.positions = {}  # key:value = index:positions
        
    def __repr__(self):
        return f"Bricks"
    
    def get_sorted_bricks(self):
        """Sort brick indexes based on their lowest z-value"""
        
        def min_z(positions):
            return min(positions, key=lambda p: p[2])[2]
        
        sorted_bricks = sorted(self.positions.keys(), key=lambda idx: min_z(self.positions[idx]))
        return sorted_bricks
        
    def move(self, brick_index):
        new_positions = []
        for x, y, z in self.positions[brick_index]:
            old_position = (x, y, z)
            new_position = (x, y, z-1)
            
            new_positions.append(new_position)
            del self.index[old_position]
            self.index[new_position] = brick_index
        
        self.positions[brick_index] = new_positions
            
    def is_movable(self, brick_index, ignore_brick_index=None):
        bricks_to_ignore = [brick_index, ignore_brick_index] if ignore_brick_index is not None else [brick_index]
        for x, y, z in self.positions[brick_index]:
            if z < 2:   # check for ground
                return False
            if (x, y, z-1) in self.index:  # check for other bricks
                if self.index[(x, y, z-1)] not in bricks_to_ignore:
                    return False
        return True
    
    def supporting_bricks(self, brick_index):
        supports = []
        for x, y, z in self.positions[brick_index]:
            if (x, y, z+1) in self.index:
                if self.index[x, y, z+1] != brick_index:
                    supports.append(self.index[(x, y, z+1)])
        return supports
    
    def settle(self):
        for brick_index in self.get_sorted_bricks():
            while self.is_movable(brick_index):
                self.move(brick_index)
                
    def safe_to_disintegrate(self):
        disintegratable_bricks = []
        for removed_brick in self.get_sorted_bricks():
            disintegratable = True
            for brick in self.get_sorted_bricks():
                if brick == removed_brick:
                    continue
                if self.is_movable(brick, removed_brick):
                    disintegratable=False
                    break
            if disintegratable:
                disintegratable_bricks.append(removed_brick)
        return set(disintegratable_bricks)
        
    def visualize(self, first=7):
        for i in range(first):
            positions = self.positions[i]
            for x,y,z in positions:
                plt.plot(x, y, z)
        
    
    


In [46]:
import numpy as np 


def my_func(input_string):
    bricks = Bricks()
    # alphabet = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
    for i, line in enumerate(input_string.splitlines()):
        start, end = line.split('~')
        
        start_position = np.array(tuple(map(int, start.split(','))))
        end_position = np.array(tuple(map(int, end.split(','))))
        
        steps = max(abs(end_position - start_position))+1
        positions = np.linspace(start_position, end_position, steps)
        for position in positions:
            bricks.index[tuple(position)] = i
        
        bricks.positions[i] = [tuple(p) for p in positions]
        
    bricks.settle()
    bricks.safe_to_disintegrate()
    
    return len(bricks.safe_to_disintegrate())
    

### Example

In [47]:
input_string = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9"""


my_func(input_string)


5

### Submission

In [48]:
with open('input/day22.txt', 'r') as file:
    input_string = file.read()

my_func(input_string)


517

## Part 2

text

### Solution

In [6]:
def my_func(input_string):
    pass


### Example

In [ ]:
input_string = """"""


my_func(input_string)


### Submission

In [ ]:
with open('input/day00.txt', 'r') as file:
    input_string = file.read()


my_func(input_string)
